In [6]:
from astroquery.simbad import Simbad
from astroquery.gaia import Gaia
from astroquery.mast import Catalogs

import numpy as np
import pandas as pd

import os
from astropy.timeseries import LombScargle
import lightkurve as lk
import matplotlib.pyplot as plt
import shutil
from astropy.utils.data import _get_download_cache_loc

In [144]:
### Getting data needed

def selective_search(tic_id):
    search_result = lk.search_lightcurve(f'TIC {tic_id}', mission='TESS')

    if not search_result:
        print(f"⚠️ No TESS light curves found for TIC {tic_id}.")
        return None
    
    try:
        selec_search = search_result[search_result.author == 'SPOC']
        exptimes = selec_search.table['exptime']
        min_index = exptimes.argmin()
        parsed_search = selec_search[min_index]
    except:
        print("No Spoc result found")
        exptimes = search_result.table['exptime']
        min_index = exptimes.argmin()
        parsed_search = search_result[min_index]
        
    return parsed_search

#downloading data and removing bad data points function

def dwnlwd(parsed_search):
    data = parsed_search.download(quality_bitmask='default').remove_nans()
    return data

#extracting data

def extracter(data):
    #.remove_outliers()
    
    time, flux = data.time.value, data.flux.value
    flux /= np.median(flux)
    time -= time[0]

    return time, flux

#calculating fourier transform

def calc_lomb_scargle(t,y):
    oversample = 10 # can be adjusted
    tmax = t.max()
    tmin = t.min()
    df = 1.0 / (tmax - tmin)
    fmin = df
    fmax = 1000 # set max freq in c/d

    freq = np.arange(fmin, fmax, df / oversample)
    model = LombScargle(t, y)
    sc = model.power(freq, method="fast", normalization="psd")

    fct = np.sqrt(4./len(t))
    amp = np.sqrt(sc) * fct * 1e6
    return freq, amp # freq in cycles per day and amp in ppm

#finding most likely peak due to rotation

def rotation_f(freq,amp,fmin,fmax):
    mask = (freq >= fmin) & (freq <= fmax)
    freq_in_range = freq[mask]
    amp_in_range = amp[mask]
    idx_max = np.argmax(amp_in_range)
    dom_freq = freq_in_range[idx_max]

    return dom_freq

#Compilation function
def f_finder(tic_id,fmin,fmax):
    parsed_search_result = selective_search(tic_id)

    if parsed_search_result == None:
        return
    
    data = dwnlwd(parsed_search_result)

    time, flux = extracter(data)

    freq, amp = calc_lomb_scargle(time, flux)

    #cycles per day
    f = rotation_f(freq,amp,fmin,fmax)
    
    return f

In [145]:
#test
test_id = 288404080

dom_freq = f_finder(test_id,0,5)

print(f"Dominant frequency: {dom_freq} cycles per day")

Dominant frequency: 1.5147723536278335 cycles per day


In [7]:
simbad = Simbad()

simbad.list_columns("basic").show_in_browser()

In [125]:
simbad = Simbad()

simbad.list_columns("mesRot").show_in_browser()

In [8]:
simbad = Simbad()

simbad.list_columns("ident").show_in_browser()

In [5]:
simbad = Simbad()

simbad.list_columns("ids").show_in_browser()

In [128]:
#querying simbad for rotation measurements

def vsini_finder(tic_id):
    name = f"TIC {tic_id}"
    
    query = f"""
        SELECT i.id, m.vsini
        FROM mesRot AS m
        JOIN ident AS i ON m.oidref = i.oidref
        WHERE i.id = '{name}'"""

    result = simbad.query_tap(query)

    vsini = np.mean(result['vsini'])

    return vsini


In [129]:
vsini_finder(test_id)

134.0

In [130]:
gaiadr3_table = Gaia.load_table('gaiadr3.gaia_source')

for column in gaiadr3_table.columns:
  print(column.name)

solution_id
designation
source_id
random_index
ref_epoch
ra
ra_error
dec
dec_error
parallax
parallax_error
parallax_over_error
pm
pmra
pmra_error
pmdec
pmdec_error
ra_dec_corr
ra_parallax_corr
ra_pmra_corr
ra_pmdec_corr
dec_parallax_corr
dec_pmra_corr
dec_pmdec_corr
parallax_pmra_corr
parallax_pmdec_corr
pmra_pmdec_corr
astrometric_n_obs_al
astrometric_n_obs_ac
astrometric_n_good_obs_al
astrometric_n_bad_obs_al
astrometric_gof_al
astrometric_chi2_al
astrometric_excess_noise
astrometric_excess_noise_sig
astrometric_params_solved
astrometric_primary_flag
nu_eff_used_in_astrometry
pseudocolour
pseudocolour_error
ra_pseudocolour_corr
dec_pseudocolour_corr
parallax_pseudocolour_corr
pmra_pseudocolour_corr
pmdec_pseudocolour_corr
astrometric_matched_transits
visibility_periods_used
astrometric_sigma5d_max
matched_transits
new_matched_transits
matched_transits_removed
ipd_gof_harmonic_amplitude
ipd_gof_harmonic_phase
ipd_frac_multi_peak
ipd_frac_odd_win
ruwe
scan_direction_strength_k1
scan_di

In [131]:
#finds gaia id (data release 3) given a tic id

def gaia_id_finder(tic_id):
    name = f"TIC {tic_id}"  # SIMBAD expects a space

    query = f"""
        SELECT i.id, gai.ids
        FROM ident AS i
        JOIN ids AS gai ON i.oidref = gai.oidref
        WHERE i.id = '{name}'"""

    result = simbad.query_tap(query)

    all_ids = result['ids'][0]  # e.g., 'TIC 288404080|Gaia DR3 6890965518204449536|* 18 Aqr|...'

    # Split the string and find the Gaia DR3 ID
    gaia_id = None
    for ident in all_ids.split('|'):
        ident = ident.strip()
        if ident.startswith('Gaia DR3'):
            gaia_id = ident.split()[-1]
            break

    return gaia_id

In [132]:
#testing gaia_id_finder

test_id = 288404080

print(gaia_id_finder(test_id))

6890965518204449536


In [ ]:
#teff from gaia in kelvin

#radius_gspphot

#teff_gspphot
#teff_gspspec

def teff_finder(gaia_id):
    query = f"""SELECT astropar.teff_gspphot as Temp FROM gaiadr3.gaia_source AS gaia JOIN gaiadr3.astrophysical_parameters AS astropar 
    ON gaia.source_id = astropar.source_id
    WHERE gaia.source_id = {gaia_id}"""
    
    job = Gaia.launch_job_async(query)

    res = job.get_results()

    return res['Temp'][0]
    

In [ ]:
#testing teff_finder

test_id = 288404080

print(teff_finder(gaia_id_finder(test_id)))

used

In [134]:
def radius_finder(tic_id):
    
    catalog_data = Catalogs.query_criteria(catalog="Tic", ID=tic_id)
    
    result = catalog_data['rad'][0]
    
    return result
    

In [135]:
print(radius_finder(test_id))

2.11098


In [140]:
#converting vsini to fsini, cycles per day

def converter(vsini,radius):
    factor = (86400 * 1000)/(6.957e8)
    circumference = 2 * np.pi * radius
    fsini = factor * (vsini/circumference)
    return fsini

In [141]:
print((86400 * 1000)/(6.957e8))

0.12419146183699871


In [142]:
def inclination(tic_id,fmin,fmax):
    f = f_finder(tic_id,fmin,fmax)
    
    vsini = vsini_finder(tic_id)
    radius = radius_finder(tic_id)
    fsini = converter(vsini,radius)
    
    i = np.arcsin(fsini/f)
    
    return i

In [143]:
test_id = 288404080

print(inclination(test_id,0,5))

156.86953879533044
0.9760582274342721


In [ ]:
#Analysis

ids = [288404080,,,,,]

for value in ids:
    print(f"Angle of inclination of star TIC {value} found to be: {inclination(test_id,0,5)} rads")